Họ tên: Ninh Duy Huy

MSSV: 19120533

# HW2: Cách thực thi song song trong CUDA

Với các GPU tương đối mới thì để biên dịch chỉ cần dùng câu lệnh: \
`nvcc tên-file.cu -o tên-file-chạy`

Nhưng trên Colab mình thường lấy được GPU khá cũ là Tesla K80 với compute capability (phiên bản phần cứng) là 3.7; để biên dịch đúng với GPU khá cũ này thì bạn cần dùng câu lệnh: \
`nvcc -arch=sm_37 tên-file.cu -o tên-file-chạy` \
Trong đó, 37 chính là compute capability của GPU Tesla K80.

Để phòng trường hợp khi làm bài bạn lấy được GPU có compute capability x.x nhưng khi chấm bài Thầy lại lấy được GPU có compute capability khác x.x, dưới đây mình sẽ có đoạn code Python để tự động lấy 2 con số ứng với compute capability của GPU và lưu vào 2 biến `major` và `minor`:


In [60]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


Một khi đã chạy đoạn code Python ở trên, để biên dịch thì bạn sẽ dùng câu lệnh: \
`nvcc -arch=sm_{major}{minor} tên-file.cu -o tên-file-chạy`

Dưới đây, khi làm bài thì bạn có thể tùy ý thêm/xóa cell. Đừng xóa mấy cell có chữ của Thầy là được.

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
import os

os.chdir('drive/MyDrive/lập trình song song/HW2')
os.getcwd()

FileNotFoundError: ignored

In [62]:
os.listdir()

['HW2.ipynb',
 'cuda-repo-ubuntu1804_10.0.130-1_amd64.deb',
 'HW2-CUDAParallelExecution.pdf',
 'HW2_P1.cu',
 'HW2_P2.cu',
 'a.out',
 'cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb',
 'a2.nvvp',
 'a1.nvvp',
 'a2.png']

## Câu 1A

In [63]:
!nvcc -arch=sm_{major}{minor} HW2_P1.cu

In [64]:
# # chạy chương trình với blocksize mặc định
!./a.out

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
****************************

Input size: 16777217

Kernel 1
Grid size: 32769, block size: 512
Kernel time = 3.445280 ms, post-kernel time = 0.103008 ms
CORRECT :)

Kernel 2
Grid size: 32769, block size: 512
Kernel time = 1.906272 ms, post-kernel time = 0.109504 ms
CORRECT :)

Kernel 3
Grid size: 32769, block size: 512
Kernel time = 1.624416 ms, post-kernel time = 0.101408 ms
CORRECT :)


-  Ta có thể thấy được thời gian chạy của 3 reduce kernel có sự khác nhau (blocksize=512)
  + kernel 1: 3.32 ms
  + kernel 2: 1.87 ms
  + kernel 3: 1.57

Ở kernel 1 các thread có thực hiện tính toán thưa thớt xen kẻ, kernel 2 thì các thread không có tính toán ta sẽ không duyệt qua hạn chế được sự phân kì, Ở kernel 3 hạn chế được sự phân kì trên wrap (vì các thread tính toán năm liền kề nhau) giúp cải thiệt tóc độ tính toán

## Câu 1B


In [65]:
# blocksize 1024
!./a.out 1024

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
****************************

Input size: 16777217

Kernel 1
Grid size: 16385, block size: 1024
Kernel time = 4.495872 ms, post-kernel time = 0.051104 ms
CORRECT :)

Kernel 2
Grid size: 16385, block size: 1024
Kernel time = 2.298688 ms, post-kernel time = 0.050336 ms
CORRECT :)

Kernel 3
Grid size: 16385, block size: 1024
Kernel time = 2.031840 ms, post-kernel time = 0.054368 ms
CORRECT :)


In [66]:
# blocksize 512
!./a.out 512

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
****************************

Input size: 16777217

Kernel 1
Grid size: 32769, block size: 512
Kernel time = 3.471200 ms, post-kernel time = 0.089088 ms
CORRECT :)

Kernel 2
Grid size: 32769, block size: 512
Kernel time = 1.907072 ms, post-kernel time = 0.088192 ms
CORRECT :)

Kernel 3
Grid size: 32769, block size: 512
Kernel time = 1.625984 ms, post-kernel time = 0.097248 ms
CORRECT :)


In [67]:
# blocksize 256
!./a.out 256

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
****************************

Input size: 16777217

Kernel 1
Grid size: 65537, block size: 256
Kernel time = 2.796096 ms, post-kernel time = 0.165440 ms
CORRECT :)

Kernel 2
Grid size: 65537, block size: 256
Kernel time = 1.595872 ms, post-kernel time = 0.181088 ms
CORRECT :)

Kernel 3
Grid size: 65537, block size: 256
Kernel time = 1.405728 ms, post-kernel time = 0.177408 ms
CORRECT :)


In [68]:
# blocksize 128
!./a.out 128

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
****************************

Input size: 16777217

Kernel 1
Grid size: 131073, block size: 128
Kernel time = 2.418560 ms, post-kernel time = 0.327200 ms
CORRECT :)

Kernel 2
Grid size: 131073, block size: 128
Kernel time = 1.438944 ms, post-kernel time = 0.330080 ms
CORRECT :)

Kernel 3
Grid size: 131073, block size: 128
Kernel time = 1.288000 ms, post-kernel time = 0.336992 ms
CORRECT :)


Block size | Grid size | Num blocks / SM | Occupancy (%) | Kernel time (ms) | Post-kernel time (ms) | Total time (ms)
--- | --- | --- | --- | --- | --- | ---
1024 | 16385 | 1 | 100 | 4.41 | 0.05 | 4.45
512 | 32769 | 2 | 100 | 3.33 | 0.09 | 3.42
256 | 65537 | 4 | 100 | 2.74 | 0.19 | 2.93
128 | 131073 | 8 | 100 | 2.34 | 0.32 | 2.66

- max blocks per SM: 16
- max warps per SM:  32
- max threads per SM: 1024

- Số lượng blocks tính bằng số lượng threads trong 1 blocks (blocksize) chia cho số lượng threads tối đa trên 1 SM
- Occupancy = (blocksize*numBlocks)/maxThreadsPerSM 

## Câu 2

In [69]:
!nvcc -arch=sm_{major}{minor} HW2_P2.cu

In [70]:
!./a.out

n =  16777217

GPU name: Tesla T4
GPU compute capability: 7.5
Processing time of all device streams: 43.095745 ms

CORRECT :)


In [71]:
!./a.out 
# !/usr/local/cuda-10.0/bin/nvprof -o a1.nvvp -f ./a.out

n =  16777217

GPU name: Tesla T4
GPU compute capability: 7.5
Processing time of all device streams: 44.841312 ms

CORRECT :)


- Khi sử dụng 1 stream thì nó sẽ như việc cộng 2 vector xong xong bình thường (load hết dữ liệu rồi mới bắt đầu cộng)

![My Image](https://raw.githubusercontent.com/ninhduyhuy24022001/LTSS_IMage/main/a1.png)

In [72]:
!./a.out 512 3
# !/usr/local/cuda-10.0/bin/nvprof -o a2.nvvp -f ./a.out 512 3

n =  16777217

GPU name: Tesla T4
GPU compute capability: 7.5
Processing time of all device streams: 37.686497 ms

CORRECT :)


- Em không biết tại sao việc copy từ host san device của em lại không thực hiện trên từng stream mà lại thực ở trên, nhưng có thể thấy được việc ovlerlap của các stream.

![My Image](https://github.com/ninhduyhuy24022001/LTSS_IMage/blob/main/a2.png?raw=true)

In [73]:
# #Uninstall the current CUDA version
# !apt-get --purge remove cuda nvidia* libnvidia-*
# !dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
# !apt-get remove cuda-*
# !apt autoremove
# !apt-get update

In [74]:
# #Download CUDA 10.0
# !wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
# #install CUDA kit dpkg
# !dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
# !sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
# !apt-get update
# !apt-get install cuda-10-0

In [75]:
# !/usr/local/cuda-10.0/bin/nvprof -o a1.nvvp -f ./a.out